In [1]:
from pdf_analyzer.config_load import load_config
from pdf_analyzer.api import API
from omegaconf import OmegaConf
from pdf_analyzer.config_load import load_config

from pdf_analyzer.api.extract.elements import extract_element_soup

In [2]:
from get_paper_metadata import get_all_paper_metadata

In [3]:
server_config = load_config("config/api/grobid-server-config.yaml")
extract_config = load_config("config/api/api-base-config.yaml")
print("SERVER_CONFIG\n"+OmegaConf.to_yaml(server_config))
print("CLOUD_CONFIG\n"+OmegaConf.to_yaml(extract_config))

base_api = API.BaseAPI(extract_config,server_config)

files = base_api.proccesed_files

SERVER_CONFIG
url:
  protocol: http
  api_domain: yordi111nas.synology.me
  port: 8070

CLOUD_CONFIG
data:
  data_dir: data/PDFs
  format: .pdf
  recursive: true
grobid:
  cache: true
  cache_dir: data/xmls
  operation_key: processFulltextDocument
  format: .grobid.tei.xml
  recursive: true

http://yordi111nas.synology.me:8070/api/isalive
GROBID server is up and running
data/xmls\nlp\Bert.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\DistillBERT.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\Dont_stop_pretraining.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\GPT-3.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\LIME.grobid.tei.xml already exist, skipping... (use --force to reprocess pdf input files)
data/xmls\nlp\LoRA.grobid.tei.xml already exist, skipping... (use --force to repro

In [4]:


from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from bs4 import BeautifulSoup
from datetime import datetime

# Definir los prefijos de los namespaces
mod = Namespace("https://w3id.org/mod#")
ns = Namespace("http://open_science.com#")

# Crear un grafo RDF
g = Graph()

# Función para convertir la fecha de formato XML a XSD
def convert_date(xml_date):
    formats = ["%d %b %Y", "%Y-%m-%d"]  # Probamos dos formatos diferentes
    for fmt in formats:
        try:
            return datetime.strptime(xml_date.text.strip(), fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    return None

# Iterar sobre los datos de los papers y agregarlos al grafo
papers_data =get_all_paper_metadata(files)
for paper in papers_data:
    paper_uri = URIRef(f"http://open_science.com#Paper/{paper['paper_id']}")
    g.add((paper_uri, RDF.type, ns.Paper))
    g.add((paper_uri, ns.title, Literal(paper['title'])))
    g.add((paper_uri, ns.abstract, Literal(paper['abstract'])))
    g.add((paper_uri, RDFS.label, Literal(paper['title'])))

    # Utilizar BeautifulSoup para analizar el elemento XML de fecha
    if paper['published_date']!=None:
        xml_date = paper['published_date']
        published_date = convert_date(xml_date)
        if published_date:
            g.add((paper_uri, ns.release_date, Literal(published_date, datatype=XSD.date)))

    # Aquí puedes agregar más relaciones según sea necesario, como autores, organizaciones, etc.

# Serializar el grafo RDF en formato Turtle
print(g.serialize(format='turtle'))

@prefix ns1: <http://open_science.com#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://open_science.com#Paper/0> a ns1:Paper ;
    rdfs:label "<title level=\"a\" type=\"main\">Exploring teachers’ confidence in addressing mental health issues in learners with Profound and Multiple Learning Difficulties (PMLD) pre and post training</title>" ;
    ns1:abstract """ 
We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a;Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering a

In [5]:
from ner.extract_ner import get_all_ners,get_all_projects
from extract_authors import get_all_author_metadata

In [6]:
from transformers import pipeline

pipe = pipeline("token-classification", model="dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
all_orgs_rel,all_orgs=get_all_ners(files,pipe)
authors_list,relation_author_paper,all_orgs,relation_author_org =get_all_author_metadata(files,all_orgs)

In [8]:
for org in all_orgs:
    org_uri = ns[f"Organization/{org['org_id']}"]
    g.add((org_uri, RDF.type, ns.Organization))
    g.add((org_uri, ns.name, Literal(org['name'])))
    g.add((org_uri, RDFS.label, Literal(org['name'])))


In [9]:
for dic in all_orgs_rel:
    # URI de la organización
    organization_uri = ns[f"Organization/{dic['org_id']}"]
    
    # URI del papel
    paper_uri = ns[f"Paper/{dic['paper_id']}"]
    
    # Agregar tripleta al grafo RDF
    g.add((paper_uri, ns.acknowledges, organization_uri))

In [ ]:
for dic in relation_author_org:
    # URI de la organización
    organization_uri = ns[f"Organization/{dic['org_id']}"]
    
    # URI del papel
    author_id = ns[f"Author/{dic['author_id']}"]
    
    # Agregar tripleta al grafo RDF
    g.add((author_id, ns.member, organization_uri))

In [ ]:
for dic in relation_author_org:
    # URI de la organización
    paper_uri = ns[f"Paper/{dic['paper_id']}"]
    
    # URI del papel
    author_id = ns[f"Author/{dic['author_id']}"]
    
    # Agregar tripleta al grafo RDF
    g.add((author_id, ns.author, paper_uri))

In [10]:
all_projects,all_projects_relation=get_all_projects(files)

In [11]:
for project in all_projects:

    project_uri = ns[f"Project/{project['project_id']}"]

    # Crear la etiqueta del proyecto
    label = f"Award {project['project_name']} {project['project_federal_id']}"

    # Agregar triples al grafo RDF
    g.add((project_uri, RDF.type, ns.Project))
    g.add((project_uri, ns.name, Literal(project['project_name'], datatype=XSD.string)))
    g.add((project_uri, RDFS.label, Literal(label, datatype=XSD.string)))
    g.add((project_uri, ns.project_federal_id, Literal(project['project_federal_id'], datatype=XSD.string)))


In [12]:
for relation in all_projects_relation:
    # URI del proyecto
    project_uri = ns[f"Project/{relation['project_id']}"]
    
    # URI del paper
    paper_uri = ns[f"Paper/{relation['paper_id']}"]
    
    # Agregar tripleta al grafo RDF
    g.add((paper_uri, ns.acknowledges, project_uri))

In [13]:
print(g.serialize(format='turtle'))

@prefix ns1: <http://open_science.com#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://open_science.com#Paper/0> a ns1:Paper ;
    rdfs:label "<title level=\"a\" type=\"main\">Exploring teachers’ confidence in addressing mental health issues in learners with Profound and Multiple Learning Difficulties (PMLD) pre and post training</title>" ;
    ns1:abstract """ 
We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a;Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering a